## Training a neural network to predict the position of body features from VR controllers

In [1]:
%matplotlib inline
from __future__ import print_function
import glob
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sbn
from sklearn.ensemble import IsolationForest
from sklearn.neural_network import MLPRegressor

### Reading the recorded data

The data recorded are stored in CSV formatted files, which we read in as pandas data frames and extract the input and output features

Note: The direction of the headset and controllers are stored in the data files as well, in the form of quaternions, with labels 'HeadsetQuaternionX', 'HeadsetQuaternionY', 'HeadsetQuaternionZ', 'HeadsetQuaternionW' and similar for LController and RController. If you are familiar with quaternion math (In a left-handed coordinate system), try to include these features in the training.

In [2]:
# Create list of data files
data_files = glob.glob('../../data/*_steamVRPositions_*')

# The input features are XYZ positions of the headset and controllers
input_features = ['Headset', 'LController', 'RController']
# The output features are XYZ positions of the front, back, elbows and knees
output_features = ['Front', 'Back', 'LeftElbow', 'RightElbow', 'LeftKnee', 'RightKnee']
# Create the labels to be read
input_labels = [x+y for x in input_features for y in 'XYZ']
output_labels = [x+y for x in output_features for y in 'XYZ']
print('Input labels are: \n', ", ".join(input_labels))
print('Output labels are: \n', ", ".join(output_labels))

# Read in the needed columns from the data files
# The data is recorded with the simulation box size as a reference, which we need to read to
# properly scale the data.
all_input_data = []
all_output_data = []
all_box_scales = []
all_player_ids = []
# Read each data file
for f in data_files:
    fname = f.split('/')[-1]
    # Keep track of the player generating the data
    if fname[0] == 'f':
        # Player Felix
        player_id = 0
    elif fname[0] == 'r':
        # Player Rob
        player_id = 1
    elif fname[0] == 'l':
        # Player Lisa
        player_id = 2
    elif fname[0] == 'h':
        # Player Helen
        player_id = 3
    elif fname[0] == 's':
        # Player Silvia
        player_id = 4
    else:
        # Random players
        player_id = 5
        
    df = pd.read_csv(f)
    # drop rows with NaN
    df = df.dropna()
    all_input_data.append(df[input_labels].values)
    all_output_data.append(df[output_labels].values)
    all_box_scales.append(df['Scale'].values)
    all_player_ids.extend([player_id]*len(df))

# Join data together as a single array
data_in = np.concatenate(all_input_data)
data_out = np.concatenate(all_output_data)
# Make box_scales a 2d-array for easier maths operations later
box_scales = np.concatenate(all_box_scales)[:,None]
player_ids = np.asarray(all_player_ids)
print('Shape of data_in:', data_in.shape)
print('Shape of data_out', data_out.shape)
print('Shape of box_scales', box_scales.shape)

Input labels are: 
 HeadsetX, HeadsetY, HeadsetZ, LControllerX, LControllerY, LControllerZ, RControllerX, RControllerY, RControllerZ
Output labels are: 
 FrontX, FrontY, FrontZ, BackX, BackY, BackZ, LeftElbowX, LeftElbowY, LeftElbowZ, RightElbowX, RightElbowY, RightElbowZ, LeftKneeX, LeftKneeY, LeftKneeZ, RightKneeX, RightKneeY, RightKneeZ
Shape of data_in: (32454, 9)
Shape of data_out (32454, 18)
Shape of box_scales (32454, 1)


### Preprocessing

When recording, people will move around the room (translation), change the direction they're facing (rotation) and the height will vary from person to person (height scaling). In addition, the positional data is recorded relative to the simulation box size (box scaling).
The machine learning algorithm will be more effective if we normalise the data to a common reference, which can be done in many different ways. The way we have chosen centers each frame such that the average of the X and Z components of the headset and controllers are zero. Then we rotate such that the head is placed on the negative X-axis with Z-component of zero (we assume the head is facing the center).

Note: If you can think of an alternative way of normalising the data, try to implement this and compare the results.

In [4]:
# Scale by VR-box size
data_in *= box_scales
data_out *= box_scales

# Shift vertically so feet are at approx y=0
# The floor_position is a known constant
floor_position = 1.0

# Only change the Y-components (indices 1, 4, ...)
data_in[:,1::3] += floor_position
data_out[:,1::3] += floor_position

# Scale by each persons height from the first frame
data_in /= data_in[0,1]
data_out /= data_in[0,1]

# Get mean x and z coordinates of head and hands
# Store as 2d-array for easier math
mean_x = np.mean(data_in[:,::3], axis=1)[:,None]
mean_z = np.mean(data_in[:,2::3], axis=1)[:,None]

# center the data
data_in[:,::3] -= mean_x
data_out[:,::3] -= mean_x
data_in[:,2::3] -= mean_z
data_out[:,2::3] -= mean_z

# Rotate head onto the negative X-axis
def rotate(array1, array2, angle):
    """ Rotate anti-clockwise around (0,1,0) by angle radians.
        X' = cos(angle) * X - sin(angle) * Z
        Z' = sin(angle) * X + cos(angle) * Z
    """
    c = np.cos(angle)
    s = np.sin(angle)
    array1[:,::3], array1[:,2::3] = (c * array1[:,::3] - s * array1[:,2::3], s * array1[:,::3] + c * array1[:,2::3])
    array2[:,::3], array2[:,2::3] = (c * array2[:,::3] - s * array2[:,2::3], s * array2[:,::3] + c * array2[:,2::3])

# Calculate the rotation angles (2d-array to make the math easier)
head_angles = np.pi - np.arctan2(data_in[:,2], data_in[:,0])[:,None]
# Rotate the input and output features
rotate(data_in, data_out, head_angles)
# Verify that the Z-component of the headset is now zero
print('Headset Z-component min and max:', data_in[:,2].min(), data_in[:,2].max())

# Since the Z-component is zero, we don't need it for training, and will ignore it from here on
data_in = np.delete(data_in, 2, axis=1)
print('Input data shape after removing axis:', data_in.shape)

Headset Z-component min and max: -1.94289029309e-16 4.23272528138e-16
Input data shape after removing axis: (32454, 8)


### Outlier removal
Due to occlusion of the sensors sometimes one of the controllers will drift off. This is fine for the input features, as the occlusion can happen during use, but it's something that we'd like to remove from the output features. Here we use the IsolationForrest method implemented in scikit-learn to classify where occlusion might have occured. (http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.IsolationForest.html)

Note: Other methods could be used as well. Take a look at http://scikit-learn.org/stable/modules/outlier_detection.html and try to use a different method.

Note: When occlusion of a sensor happens, each drift should be constant in a specific direction (a drift at a later time might happen with a different rate in a different direction though). Try to detect outliers by using this information.

In [ ]:
# Our estimations of the number of outliers in the output features
outlier_fraction = 0.05

# Construct and fit the isolation forrest
clf = IsolationForest(max_samples=1.0, contamination=outlier_fraction)
clf.fit(data_out)

# Predict which frames have outliers
prediction = clf.predict(data_out)

# prediction will be -1 if the point is an outlier and 1 if it is not.
# We convert this output to a boolean array
inliers = np.asarray((prediction+1) // 2, dtype=bool)

# Only retain the frames that isn't detected as outliers
n_data_before = data_in.shape[0]
data_in_pruned = data_in[inliers]
data_out_pruned = data_out[inliers]
player_ids_pruned = player_ids[inliers]
n_data_after = data_in_pruned.shape[0]
print('Removed %d out of %d data points (%.1f %%)' % (n_data_before - n_data_after, n_data_before,(1-n_data_after/n_data_before)*100 ))

Removed 1623 out of 32454 data points (5.0 %)


We can visualise the data before and after outlier removal to see if it looks like we would expect.

Note: Try to change the outlier_fraction and view the effect.

In [ ]:
def plot_outliers(data, data_pruned, labels):
    n_features = len(labels)
    fig, axes = plt.subplots(n_features, 1)
    
    fig.set_size_inches(6, 3.5*n_features)
    for i in range(n_features):
        min_value = min(data[:,i])
        max_value = max(data[:,i])
        binwidth = 0.1
        bins = np.arange(min(data[:, i]), max(data[:, i]) + binwidth, binwidth)
        axes[i].set_yscale('log') # Set logarithmic y axis
        axes[i].set_title('%s'%labels[i])
        axes[i].set_ylabel('Probability density')
        axes[i].hist(data[:, i], bins=bins, stacked=False)
        axes[i].hist(data_pruned[:, i], bins=bins, stacked=False)
    
    plt.show()

plot_outliers(data_out, data_out_pruned, output_labels)


### Split data into training, test and validation data
To avoid overfitting, we split the data into a training set, validation set and test set. This is known as cross validation. The training set is what the neural network is actually trained on. The hyper-parameters of the neural network (learning rate, regularisation, nodes in each layer etc.) can then be selected to optimise the predictive performance on the validation set, which has not been included in the training. This makes sure that the prediction of the neural network is transferable ie. still performs well on people not included in the training.
Finally, the test set is used to measure the performance of the selected model.

To keep things simple we use Felix' data as validation set and Silvia's data as test set. More robust ways of performing cross validation can be used, which you can read more about at http://scikit-learn.org/stable/modules/cross_validation.html and http://scikit-learn.org/stable/tutorial/statistical_inference/model_selection.html

In [ ]:
train_in = data_in_pruned[np.where((player_ids_pruned != 0) & (player_ids_pruned != 4))]
train_out = data_out_pruned[np.where((player_ids_pruned != 0) & (player_ids_pruned != 4))]
val_in = data_in_pruned[np.where(player_ids_pruned == 0)]
val_out = data_out_pruned[np.where(player_ids_pruned == 0)]
test_in = data_in_pruned[np.where(player_ids_pruned == 4)]
test_out = data_out_pruned[np.where(player_ids_pruned == 4)]
print('Number of training points:', train_in.shape[0])
print('Number of validation points:', val_in.shape[0])
print('Number of test points:', test_in.shape[0])

### Training the neural network
We use the multilayer perceptron implementation in scikit-learn: http://scikit-learn.org/stable/modules/generated/sklearn.neural_network.MLPRegressor.html

When more than a few hyperparameters affects the performance of the model, it becomes impossible to optimise the hyperparameters with a standard grid search. Instead the optimisation is typically done with genetic algorithms or gaussian processes, however any thorough search will be very costly, so here you can just change the hyper parameters by hand.

Note: Here we predict all 18 output features with one network. You can definately get better results by training a network a subset of the features. E.g. only predicting the left elbow or even just the X-coordinate of the left elbow.

Note: In principle we have twice as many data points if we mirror the body, such that left hand becomes right and vice versa. Try to implement this and see if it improved the predictions

In [ ]:
estimator = MLPRegressor(alpha = 1e-4 , hidden_layer_sizes=(64,64), batch_size=2048, early_stopping = True,
                         max_iter = 10000, learning_rate_init = 0.01, learning_rate='adaptive')
estimator.fit(train_in, train_out)
predicted_out = estimator.predict(val_in)
val_score = estimator.score(val_in, val_out)
score.append(val_score)
test_score = np.median(score)
print('Pearson correlation on test data: %.3f' % test_score)

Once we're happy with our choice of hyper parameters, we can include the validation set in the training as well for the final prediction

In [ ]:
train_val_in = data_in_pruned[np.where(player_ids_pruned != 4)]
train_val_out = data_out_pruned[np.where(player_ids_pruned != 4)]
estimator.fit(train_val_in, train_val_out)
predicted_out = estimator.predict(test_in)

Scatter plots comparing true and predicted values of (x,y,z), here for LeftElbow, indicating the machine learning performance.

In [ ]:
plt.plot(test_out[:,6],test_out[:,6],'k')
plt.scatter(test_out[:,6],predicted_out[:,6],s=0.5) # Predicted vs true of X coordinates
plt.xlabel("True X")
plt.ylabel("Predicted X")
plt.show()

In [ ]:
plt.plot(test_out[:,7],test_out[:,7], 'k')
plt.scatter(test_out[:,7],predicted_out[:,7],s=0.5) # Predicted vs true of X coordinates
plt.xlabel("True X")
plt.ylabel("Predicted X")
plt.show()

In [ ]:
plt.plot(test_out[:,8],test_out[:,8], 'k')
plt.scatter(test_out[:,8],predicted_out[:,8],s=0.5) # Predicted vs true of X coordinates
plt.xlabel("True X")
plt.ylabel("Predicted X")
plt.show()

Make violin plots showing the distribution of error for each feature in Euclidean space

In [ ]:
# list of lists of distances between predicted and target for each target position
targets_dist = [[] for i in range(test_out.shape[1] // 3)]
for test, pred in zip(test_out, predicted_out): 
    for i in range(len(test) // 3): 
        test_vec = np.array([test[i * 3 + j] for j in range(3)])
        pred_vec = np.array([pred[i * 3 + j] for j in range(3)])
        dist = np.linalg.norm(test_vec - pred_vec)
        targets_dist[i].append(dist)
# convert into numpy array
targets_dist = np.array(targets_dist)
print(targets_dist.shape)

In [ ]:
sbn.set(font_scale=1.2)
# Scale by silvias height
scaled_dist = targets_dist * 172
df = pd.DataFrame(scaled_dist.T, columns=x)
meltdf = df.melt(var_name='Target', value_name='Error (cm)')
ax = sbn.violinplot(x=meltdf['Target'], y=meltdf['Error (cm)'])
ax.set_xticklabels(ax.get_xticklabels(),rotation=30)

### Render predicted positions using Unity renderer.

In [ ]:
sys.path.append('../rendering')
import AvatarServer

In [ ]:
try:
    del server
except:
    print('No server to delete') 

In [ ]:
server = AvatarServer.AvatarServer(port=54321)

Connect to the client. This will block until a client connect.
Then send all the features, targets and predicted. 

In [ ]:
server.connect_to_client()

# Add back in the Z-component of the head that we removed earlier
full_test_in = np.zeros((test_in.shape[0],test_in.shape[1]+1))
full_test_in[:,:2] = test_in[:,:2]
full_test_in[:,3:] = test_in[:,2:]

for f, t, p in zip(full_test_in, test_out, predicted_out):
    # TODO - Wrap this up in a nice method.\n",
    feature_dict = AvatarServer.generate_dictionary_for_data(f, input_labels)
    target_dict = AvatarServer.generate_dictionary_for_data(t, output_labels)
    pred_dict = AvatarServer.generate_dictionary_for_data(p, output_labels)
    message = AvatarServer.merge_dictionaries(feature_dict, target_dict, pred_dict)
    server.send_object(message)

In [ ]:
server.close_connection()